In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import gc
import os
import time
import random
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm


# --------------------
# Paths
# --------------------
SUBMISSION_DIR = "/kaggle/working"
os.makedirs(SUBMISSION_DIR, exist_ok=True)
BEST_MODEL_PATH = os.path.join(SUBMISSION_DIR, "best_bilstm_stream.pth")

# --------------------
# Device
# --------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# --------------------
# Hyperparameters
# --------------------
patch_size = 150_000
N_sub_patches = 150

# ★ メモリ対策: データを小分けにするサイズ (500万行ずつ処理)
CHUNK_SIZE = 50_000_000 
overlap_rate = 0.2

# ★ 3位チームの裏技: TTF調整用の魔法の値
BEST_Y = 12.625

# --------------------
# Helper Function: Feature Extraction
# --------------------
def extract_features(x, N_sub):
    # x: (patch_size,) -> float32
    L_sub = len(x) // N_sub
    x = x[:N_sub * L_sub].reshape(N_sub, L_sub)
    
    # 高速化のため float32 で計算
    mean = x.mean(axis=1)
    std = x.std(axis=1)
    min_ = x.min(axis=1)
    max_ = x.max(axis=1)
    q25 = np.quantile(x, 0.25, axis=1)
    q50 = np.quantile(x, 0.50, axis=1)
    q75 = np.quantile(x, 0.75, axis=1)
    iqr = q75 - q25
    
    return np.stack([mean, std, min_, max_, q25, q50, q75, iqr], axis=1)

# --------------------
# Main: Streaming Processing (ここが心臓部)
# --------------------
rootpath = "../input/LANL-Earthquake-Prediction/"
csv_path = rootpath + "train.csv"

# 結果を格納するリスト
all_X = []
all_Y = []
all_earthquake_ids = []  # ★ 追加: 各パッチがどの地震サイクルか記録

# 正規化用のパラメータ（初期値）
train_mean = 0
train_std = 1
target_min = 0
target_max = 1

# CSVを「イテレータ」として開く（まだデータは読み込まれない）
# dtypeを指定してメモリ節約
reader = pd.read_csv(
    csv_path, 
    chunksize=CHUNK_SIZE,
    dtype={"acoustic_data": np.int16, "time_to_failure": np.float32}
)

print("Starting Chunk Processing...")

earthquake_id = 0  # 地震サイクルのカウンター
prev_y_val = None  # 前のy_valを記録

for chunk_idx, df_chunk in enumerate(tqdm(reader)):
    
    # 波形とターゲットを取得
    x_chunk = df_chunk["acoustic_data"].values.astype(np.float32)
    y_chunk = df_chunk["time_to_failure"].values.astype(np.float32)
    
    # -------------------------------------------------------
    # 最初のチャンクを使って、正規化パラメータを決める（近似）
    # 全データをなめると時間がかかるため、最初の5000万行で推定する
    # -------------------------------------------------------
    if chunk_idx == 0:
        train_mean = x_chunk.mean()
        train_std = x_chunk.std()
        target_min = y_chunk.min()
        target_max = y_chunk.max() # 注: 全体のmaxではないが、実用上大きな問題ではない
        print(f"Stats estimated: Mean={train_mean:.4f}, Std={train_std:.4f}")

    # 正規化 (In-place)
    x_chunk -= train_mean
    x_chunk /= train_std
    
    # パッチ生成ループ
    step = int(patch_size * (1 - overlap_rate))
    
    chunk_feats = []
    chunk_targets = []
    chunk_eq_ids = []  # ★ 追加
    
    for i in range(0, len(x_chunk) - patch_size, step):
        # ターゲット（パッチの最後）
        y_val = y_chunk[i + patch_size]
        
        # ★ 地震サイクルの境界検出（TTFが増加したらリセット）
        if prev_y_val is not None and y_val > prev_y_val:
            earthquake_id += 1
            print(f"  → Earthquake cycle {earthquake_id} detected")
        prev_y_val = y_val
        
        # 地震発生直後の不連続点を含むパッチを除外
        if y_val > y_chunk[i]:
            continue
            
        # 特徴量抽出
        x_raw = x_chunk[i : i + patch_size]
        feats = extract_features(x_raw, N_sub_patches)
        
        chunk_feats.append(feats)
        chunk_targets.append(y_val)
        chunk_eq_ids.append(earthquake_id)  # ★ 追加
    
    # 結果をリストに追加
    if len(chunk_feats) > 0:
        all_X.append(np.array(chunk_feats))
        all_Y.append(np.array(chunk_targets))
        all_earthquake_ids.append(np.array(chunk_eq_ids))  # ★ 追加
    
    # メモリ解放！
    del df_chunk, x_chunk, y_chunk, chunk_feats, chunk_targets, chunk_eq_ids
    gc.collect()

print("Processing complete. Concatenating...")

# リストを結合して1つのnumpy配列にする
X_final = np.concatenate(all_X, axis=0).astype(np.float32)
Y_final = np.concatenate(all_Y, axis=0).astype(np.float32)
earthquake_ids = np.concatenate(all_earthquake_ids, axis=0)  # ★ 追加

print(f"Total earthquake cycles detected: {earthquake_ids.max() + 1}")

# --------------------
# ★ 3位チームの裏技: TTF調整
# --------------------
print(f"\nApplying 3rd place TTF adjustment (BEST_Y={BEST_Y})...")

Y_adjusted = np.zeros_like(Y_final)

for eq_id in range(earthquake_ids.max() + 1):
    # この地震サイクルのマスク
    mask = (earthquake_ids == eq_id)
    segment_count = mask.sum()
    
    if segment_count == 0:
        continue
    
    # 各セグメントの調整TTFを計算
    # BEST_Y から線形に減少 (BEST_Y → 0)
    segment_indices = np.arange(segment_count)
    adjusted_ttf = BEST_Y - (BEST_Y / segment_count) * segment_indices
    
    Y_adjusted[mask] = adjusted_ttf
    
    if eq_id < 3:  # 最初の3サイクルだけ表示
        original_max = Y_final[mask].max()
        print(f"  Earthquake {eq_id}: {segment_count} segments, "
              f"Original max TTF: {original_max:.2f}s → Adjusted: {BEST_Y:.3f}s")

Y_final = Y_adjusted

# Yの正規化 (MinMax)
Y_final = (Y_final - Y_final.min()) / (Y_final.max() - Y_final.min())

print(f"\nFinal Data Shape: X={X_final.shape}, Y={Y_final.shape}")
print(f"Memory Usage of X: {X_final.nbytes / 1024**2:.2f} MB")
print(f"Adjusted TTF range: [{Y_final.min():.4f}, {Y_final.max():.4f}]")

In [ ]:
batch_size = 32
num_epochs = 1000 # 特徴量が軽いのでエポック回せます
learning_rate = 1e-4
hidden_size = 128
num_layers = 2
dropout = 0  # Dropout率

# --------------------
# Global Seed Control
# --------------------
SEED = 42  # ★ ここを変更するだけで全体のシードが変わります

# --------------------
# Reproducibility Settings
# --------------------
def set_seed(seed):
    """GPU使用時でも再現性を保証する設定"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # multi-GPU対応
    
    # CuDNNの決定論的動作を有効化
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # Python hash seedも固定（環境変数として設定推奨）
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(SEED)

# --------------------
# Dataset & Model (軽量版)
# --------------------
class SimpleDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]), torch.tensor(self.y[idx])

# DataLoader用のworker初期化関数（再現性のため）
def worker_init_fn(worker_id):
    """各DataLoaderワーカーでシードを固定"""
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# Split
indices = np.arange(len(X_final))
np.random.seed(SEED)  # ★ SEED変数を使用
np.random.shuffle(indices)
split = int(0.9 * len(indices))

train_ds = SimpleDataset(X_final[indices[:split]], Y_final[indices[:split]])
valid_ds = SimpleDataset(X_final[indices[split:]], Y_final[indices[split:]])

train_loader = DataLoader(
    train_ds, 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=2,
    worker_init_fn=worker_init_fn,
    generator=torch.Generator().manual_seed(SEED)  # ★ SEED変数を使用
)
valid_loader = DataLoader(
    valid_ds, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=2,
    worker_init_fn=worker_init_fn
)

# --------------------
# モデル定義: 重み付きプーリング（Attention）
# --------------------
class BiLSTM(nn.Module):
    """BiLSTM with Weighted Pooling Attention"""
    def __init__(self, input_size, hidden_size, num_layers, dropout=0.3):
        super().__init__()
        # LSTM層
        self.lstm = nn.LSTM(
            input_size, 
            hidden_size, 
            num_layers, 
            batch_first=True, 
            bidirectional=True, 
            dropout=dropout if num_layers > 1 else 0  # 1層の場合はdropout不要
        )
        
        # Attention層（各時刻の重要度を計算）
        # 入力: (batch, seq_len, hidden*2) → 出力: (batch, seq_len, 1)
        self.attention = nn.Linear(hidden_size * 2, 1)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
        # 最終予測層
        self.fc = nn.Linear(hidden_size * 2, 1)
    
    def forward(self, x):
        # x: (batch, seq_len, input_size) = (batch, 150, 8)
        
        # LSTM処理
        lstm_out, _ = self.lstm(x)
        # lstm_out: (batch, seq_len, hidden*2) = (batch, 150, 256)
        
        # Attention weights計算
        attn_scores = self.attention(lstm_out)
        # attn_scores: (batch, seq_len, 1) = (batch, 150, 1)
        
        # Softmaxで確率分布に変換（各時刻の重要度）
        attn_weights = torch.softmax(attn_scores, dim=1)
        # attn_weights: (batch, seq_len, 1) = (batch, 150, 1)
        
        # 重み付き和（重要な時刻を重視して集約）
        context = torch.sum(attn_weights * lstm_out, dim=1)
        # context: (batch, hidden*2) = (batch, 256)
        
        # Dropout + 最終予測
        out = self.dropout(context)
        out = self.fc(out).squeeze(1)
        # out: (batch,)
        
        return out

# モデル初期化
model = BiLSTM(8, hidden_size, num_layers, dropout=dropout).to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
criterion = nn.L1Loss()

# Training Loop
best_val = float("inf")

print("Start Training...")
print(f"Model: BiLSTM with Weighted Pooling Attention")
print(f"Parameters: hidden={hidden_size}, layers={num_layers}, dropout={dropout}")

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
    train_loss /= len(train_ds)
    
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            val_loss += criterion(pred, y).item() * x.size(0)
    val_loss /= len(valid_ds)
    
    print(f"Epoch {epoch+1}/{num_epochs} | Train: {train_loss:.4f} | Valid: {val_loss:.4f}")
    
    if val_loss < best_val:
        best_val = val_loss
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print(f"  → Best model saved! (Val Loss: {best_val:.4f})")

print("Done.")

In [ ]:
# --------------------
# Inference
# --------------------
import os
import pandas as pd
import numpy as np
import torch

# モデルの準備
model = BiLSTM(8, hidden_size, num_layers, dropout=dropout).to(device)
model.load_state_dict(torch.load(BEST_MODEL_PATH))
model.eval()

# 提出用ファイルの読み込み
sample_submission = pd.read_csv(rootpath + "sample_submission.csv")
SUBMISSION_PATH = "/kaggle/working/submission.csv"

# テストデータの読み込みと予測
test_preds = []

print("Starting Inference...")

# ★重要: sample_submissionの順序通りにファイルを読み込む
for seg_id in tqdm(sample_submission["seg_id"]):
    # ファイルパスを作成
    file_path = os.path.join(rootpath, "test", seg_id + ".csv")
    
    # 読み込み
    df = pd.read_csv(file_path, dtype={"acoustic_data": np.int16})
    x = df["acoustic_data"].values.astype(np.float32)
    
    # 1. 正規化 (学習時と同じパラメータを使用！)
    x -= train_mean 
    x /= train_std
    
    # 2. 特徴量抽出 (学習時と同じ関数を使用)
    feats = extract_features(x, N_sub_patches) # shape: (150, 8)
    
    # 3. テンソル化 (バッチ次元を追加: [1, 150, 8])
    x_tensor = torch.tensor(feats).unsqueeze(0).to(device)
    
    # 4. 予測
    with torch.no_grad():
        pred = model(x_tensor).item()
    
    test_preds.append(pred)

# 結果を格納
# ★ 3位チームの逆正規化: BEST_Yベースで復元
test_preds = np.array(test_preds)

# 正規化を解除 (0-1 → 0-BEST_Y)
test_preds = test_preds * BEST_Y

print(f"\nPrediction statistics:")
print(f"  Min: {test_preds.min():.4f}")
print(f"  Max: {test_preds.max():.4f}")
print(f"  Mean: {test_preds.mean():.4f}")
print(f"  (All values should be in range [0, {BEST_Y}])")

sample_submission["time_to_failure"] = test_preds
sample_submission.to_csv(SUBMISSION_PATH, index=False)

print(f"\nSubmission saved to: {SUBMISSION_PATH}")
print(sample_submission.head())